In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

<ipython-input-5-16cdb8520be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-7-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-8-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

<ipython-input-11-9f68b5950da1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [12]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [13]:
import gensim

In [14]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [17]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [18]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [19]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [20]:
model.build_vocab(story)

In [21]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5876298, 6212140)

In [22]:
len(model.wv.index_to_key)

31845

In [23]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [24]:
document_vector(df['review'].values[0])

array([-0.21371542,  0.48868963,  0.20875657,  0.20185682, -0.12335995,
       -0.58027124,  0.2233212 ,  0.9200318 , -0.40173453, -0.2764742 ,
       -0.25780752, -0.44809002,  0.12072115,  0.17688082,  0.21239223,
       -0.12469161,  0.00257577, -0.270651  , -0.07095958, -0.6796613 ,
        0.0221496 ,  0.2730466 ,  0.06396337, -0.2625662 , -0.32296145,
        0.00660021, -0.3029931 , -0.01611957, -0.30990994,  0.02985825,
        0.37421843,  0.02418795,  0.16519529, -0.24125536, -0.15532859,
        0.42854196,  0.07943284, -0.3432792 , -0.24922109, -0.7743544 ,
        0.13895945, -0.26802048,  0.01529006, -0.08301222,  0.47721025,
       -0.12222989, -0.21382254, -0.07215265,  0.10234919,  0.30034816,
        0.09014366, -0.33833778, -0.46383843, -0.12966172, -0.11911561,
        0.18087403,  0.22625208,  0.0489675 , -0.36511818,  0.08281382,
        0.06946047,  0.05481217,  0.04106341, -0.04141892, -0.43441468,
        0.23500443,  0.04896212,  0.15168123, -0.39654586,  0.26

In [25]:
from tqdm import tqdm

In [26]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [03:41<00:00, 45.16it/s]


In [27]:
X = np.array(X)

In [28]:
X[0]

array([-0.21371542,  0.48868963,  0.20875657,  0.20185682, -0.12335995,
       -0.58027124,  0.2233212 ,  0.9200318 , -0.40173453, -0.2764742 ,
       -0.25780752, -0.44809002,  0.12072115,  0.17688082,  0.21239223,
       -0.12469161,  0.00257577, -0.270651  , -0.07095958, -0.6796613 ,
        0.0221496 ,  0.2730466 ,  0.06396337, -0.2625662 , -0.32296145,
        0.00660021, -0.3029931 , -0.01611957, -0.30990994,  0.02985825,
        0.37421843,  0.02418795,  0.16519529, -0.24125536, -0.15532859,
        0.42854196,  0.07943284, -0.3432792 , -0.24922109, -0.7743544 ,
        0.13895945, -0.26802048,  0.01529006, -0.08301222,  0.47721025,
       -0.12222989, -0.21382254, -0.07215265,  0.10234919,  0.30034816,
        0.09014366, -0.33833778, -0.46383843, -0.12966172, -0.11911561,
        0.18087403,  0.22625208,  0.0489675 , -0.36511818,  0.08281382,
        0.06946047,  0.05481217,  0.04106341, -0.04141892, -0.43441468,
        0.23500443,  0.04896212,  0.15168123, -0.39654586,  0.26

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [30]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [33]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7636454682023035